This notebook contains code for the implementation of VGG-16 architecture from the paper 'Image Based Tomato Leaf Disease Detection' based on the network information and values provided by the authors in the paper.

The VGG-16 architecture is used as a baseline model in my thesis, 'A computer vision approach for the automatic blight disease detection in potato and tomato plants'.

In [1]:
!pip install torchsummary

In [2]:
import os                                      # for working with files
import sys
import shap                                    # for checking feature importances
import torch                                   # Pytorch module 
import shutil
import optuna
import warnings
import itertools
import numpy as np                             # for numerical computationss
import pandas as pd                            # for working with dataframes
import torch.nn as nn                          # for creating  neural networks
from PIL import Image                          # for checking images
import matplotlib.pyplot as plt                # for plotting informations on graph and images using tensors
import torch.nn.functional as F                # for functions for calculating loss
from torchsummary import summary               # for getting the summary of our model
from torchvision.models import vgg16
from torchvision.models import resnet50
from torchvision.utils import make_grid        # for data checking
from torch.utils.data import DataLoader        # for dataloaders 
import torchvision.transforms as transforms    # for transforming images into tensors 
from torchvision.datasets import ImageFolder   # for working with classes and images

%matplotlib inline

### Data exploration!

In [3]:
os.listdir('/kaggle/input/dataset/idata/Image Dataset/ImageDataset/')

In [4]:
data_dir = '/kaggle/input/dataset/idata/Image Dataset/ImageDataset/'

In [5]:
# print(f"Number of image directories are {len(os.listdir(data_fpath))+len(os.listdir('/kaggle/input/newds/ImageDataset_new/ImageDataset_new/'))}\n")
print('Number of unique plants are 2, potato and tomato\n')
print('Number of diseases are 4, early and late blight disease for tomato, early and late blight for potato\n')

In [6]:
data_dir

In [7]:
train_dir = data_dir + "train/"
valid_dir = data_dir + "valid/"
# test_dir
diseases_tr = os.listdir(train_dir)
diseases_va = os.listdir(valid_dir)


In [8]:
valid_dir

In [9]:
diseases_tr

In [10]:
folder = [i for i in os.listdir(valid_dir)]
folder

In [11]:
plants = []
NumberOfDiseases = 0
for plant in diseases_tr:
    if plant.split('___')[0] not in plants:
        plants.append(plant.split('___')[0])
    if plant.split('_')[1] != 'healthy':
        NumberOfDiseases += 1

In [12]:
# Number of images for each clas in the training data
nums_train = {}
for folder in sorted(os.listdir(f"{data_dir}/train")):
    nums_train[folder] = len(os.listdir(f"/{data_dir}/train/{folder}"))
    
# converting the nums dictionary to pandas dataframe passing index as plant name and number of images as column

img_per_training_class = pd.DataFrame(nums_train.values(), index=nums_train.keys(), columns=["no. of images"])
img_per_training_class

In [13]:
# Number of images for each clas in the training data
nums_valid = {}
for folder in sorted(os.listdir(valid_dir)):
    nums_valid[folder] = len(os.listdir(f"{valid_dir}{folder}"))
    
# converting the nums dictionary to pandas dataframe passing index as plant name and number of images as column

img_per_valid_class = pd.DataFrame(nums_valid.values(), index=nums_valid.keys(), columns=["no. of images"])
img_per_valid_class

#changes the text on matplotlib plots to the computer modern font style
from matplotlib import font_manager
font_path = '../input/compumodern/cmu.serif-roman.ttf'
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = prop.get_name()

In [14]:
# plotting number of images available for each class
index = [n for n in range(6)]
plt.figure(figsize=(8, 6))
plt.bar(index, [n for n in nums_train.values()], color='#8528B0', width=0.7, align='center')
plt.xlabel('Classes', fontsize=15)
plt.ylabel('No of images', fontsize=15)
plt.xticks(index, [key for key in nums_train.keys()], fontsize=15, rotation=90)
plt.title('Images per class for training dataset', fontsize=15)

#plt.savefig('/kaggle/working/number_imgs_training.png', dpi=600, bbox_inches="tight")  
plt.show()

In [15]:
# plotting number of images available for each class
index = [n for n in range(6)]
plt.figure(figsize=(8, 6))
plt.bar(index, [n for n in nums_valid.values()], color='#8528B0', width=0.7) 
plt.xlabel('Classes', fontsize=15)
plt.ylabel('No of images', fontsize=15)
plt.xticks(index, [key for key in nums_valid.keys()], fontsize=15, rotation=90)
plt.title('Images per class for validation dataset', fontsize=15)
# plt.tight_layout()
#plt.savefig('/kaggle/working/number_imgs_validation.png', dpi=600, bbox_inches="tight")  
plt.show()

### Data Augmentation

The data has already been augmented. see https://github.com/Alyeko/potato-tomato-blight-disease-detection

### Images available for training

In [16]:
n_train = 0
for value in nums_train.values():
    n_train += value
print(f"There are {n_train} images for training")

In [17]:
n_valid = 0
for value in nums_valid.values():
    n_valid += value
print(f"There are {n_valid} images for validation")

### Checking if here are non img files in the training data folder


In [18]:
folds = [folder for folder in os.listdir(train_dir)]
folds

In [19]:
for i in folds:
    for img in os.listdir(train_dir+i):
        if not img.endswith('.JPG'):
            print('yes!')

In [20]:
for i in folds:
    for img in os.listdir(valid_dir+i):
        if not img.endswith('.JPG'):
            print('yes!')

In [21]:
data_dir

In [22]:
print(f"There are {len(os.listdir('/kaggle/input/dataset/idata/Image Dataset/test_data/test'))} images for test")

In [23]:
print(f"Training dir: {os.listdir('/kaggle/input/dataset/idata/Image Dataset/ImageDataset/')}")
print(f"All: {os.listdir('/kaggle/input/dataset/idata/Image Dataset')}")

In [24]:
test_dir = '/kaggle/input/dataset/idata/Image Dataset/test_data/'
# print(f"There are {len(os.listdir('/kaggle/input/newds/ImageDataset_new/ImageDataset_new/test_data'))} images for training")
os.listdir(test_dir)

In [25]:
for img in os.listdir(test_dir+'test'):
        if not img.endswith('.JPG'):
            print('Yes! I knew it!')

### Data Preparation for training 

In [26]:
print(train_dir)
print(valid_dir)

code for for creating the VGG-16 network is adapted from https://www.kaggle.com/code/carloalbertobarbano/vgg16-transfer-learning-pytorch/notebook

In [27]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()  

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

In [58]:
vgg16 = models.vgg16_bn()
# vgg16.load_state_dict(torch.load("../input/vgg16bn/vgg16_bn.pth"))
print(vgg16.classifier[6].out_features) # 1000 


# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False

# Newly created modules have require_grad=True by default
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, 6)]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
print(vgg16)

In [59]:
# getting summary of the model
INPUT_SHAPE = (3, 224, 224)
print(summary(vgg16.cuda(), (INPUT_SHAPE)))

copy.deepcopy(vgg16.state_dict())

In [60]:
data_dir

vgg16 = train_model(vgg16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=2)
torch.save(vgg16.state_dict(), 'VGG16.pt')

#### The code just below moves images from the test dir to their corresponding test folders 

In [30]:
###Creating a new test dir bcause there was an svn file or folder found in the test dir
os.mkdir('../test_data')
os.mkdir('../test_data/test')

In [31]:
test_dir_old = test_dir
test_dir_new = '../test_data'
print(test_dir_old)
print(test_dir_new)

In [32]:
print(test_dir_old)
print(test_dir_new)

In [33]:
os.mkdir('../testy/')

In [34]:
test_dir_neww = "../testy/"
os.mkdir(test_dir_neww+'test')

In [35]:
# os.listdir(test_dir_neww+'test') #empty for now
folders = [folder for folder in os.listdir(data_dir+'train')] 
for folder in folders:
    os.mkdir(f"{test_dir_neww}test/{folder}")

In [36]:
os.listdir(test_dir_neww+'test') 

In [37]:
###Moving file from old test dir to new test dir
num_moved = 0
for img in os.listdir(test_dir_old+'/test'):
    if img.endswith('.JPG'):
        theclass = img.split('_')[0] + '_' + img.split('.')[0].split('_')[1]
        #print(theclass)
        shutil.copy(f"{test_dir_old+'/test/'}{img}", f"{test_dir_neww+'/test/'+theclass+'/'+img}")
        num_moved += 1
    elif img.endswith('svn'):
        print('not going to move you!')
print(f"Number of files moved: {num_moved}")

In [38]:
len(os.listdir(test_dir_neww+'test/'+'potato_late') )

###Moving file from old test dir to new test dir
num_moved = 0
for img in os.listdir(test_dir_old+'test'):
    if img.endswith('.JPG'):
        shutil.copy(f"{test_dir_old+'test/'}{img}", f"{test_dir_new+'/test/'}{img}")
        num_moved += 1
    else:
        print('not going to move you!')
print(f"Number of files moved: {num_moved}")

In [39]:
len(os.listdir('../test_data/test')) #files have been moved

In [40]:
test_images = []
for tclass in folders:
    for img in os.listdir('../testy/test/' +  tclass):
        test_images.append(img)
        
test_images = sorted(test_images)

#Testing model on test data
test = ImageFolder(f"{test_dir_neww}/test", transform=transforms.Compose(
                                            [transforms.Resize([256, 256]),
                                            transforms.ToTensor()]))

In [ ]:
test

test_images = sorted(os.listdir(test_dir_neww + '/test')) # since images in test folder are not in alphabetical order
#test_images

In [ ]:
len(test)

In [ ]:
os.listdir('../testy/test')

In [61]:
data_dir
TRAIN = 'train'
VAL = 'valid'
TEST = 'test'

# VGG-16 Takes 224x224 images as input, so we resize all of them
data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])}
#     TEST: transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#     ])
# }

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL]#, TEST]
}

test = ImageFolder(f"{test_dir_neww}/test", transform=transforms.Compose(
                                            [transforms.Resize(256),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor(),]))

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=32,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, VAL]#, TEST]
}


test_dataloader = torch.utils.data.DataLoader(test, 
                                            batch_size=32,
                                            shuffle=True, num_workers=4)

# test_loader_r = DeviceDataLoader(test_loader_r, device)
# test_loader_r

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL]}#, TEST]}
dataset_sizes_test = {'test':len(test)}

for x in [TRAIN, VAL]:#, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))

print(f"Loaded {len(test)} images under test")
print("Classes: ")
class_names = image_datasets[TRAIN].classes
print(image_datasets[TRAIN].classes)

In [62]:
len(test)

In [63]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    # plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes = next(iter(dataloaders[TRAIN]))
show_databatch(inputs, classes)

In [64]:
def visualize_model(vgg16, num_images=6):
    was_training = vgg16.training
    
    # Set model for evaluation
    vgg16.train(False)
    vgg16.eval() 
    
    images_so_far = 0

    for i, data in enumerate(test_dataloader):
        inputs, labels = data
        size = inputs.size()[0]
        
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)
        
        outputs = vgg16(inputs)
        
        _, preds = torch.max(outputs.data, 1)
        predicted_labels = [preds[j] for j in range(inputs.size()[0])]
        
        print("Ground truth:")
        show_databatch(inputs.data.cpu(), labels.data.cpu())
        print("Prediction:")
        show_databatch(inputs.data.cpu(), predicted_labels)
        
        del inputs, labels, outputs, preds, predicted_labels
        torch.cuda.empty_cache()
        
        images_so_far += size
        if images_so_far >= num_images:
            break
        
    vgg16.train(mode=was_training) # Revert model back to original training state

In [65]:
def eval_model(vgg16, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(test_dataloader)
    print("Evaluating model")
    print('-' * 10)
    
    for i, data in enumerate(test_dataloader):
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)

        vgg16.train(False)
        vgg16.eval()
        inputs, labels = data

        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        outputs = vgg16(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.item()#loss.data[0]
        acc_test += torch.sum(preds == labels.data)

        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
        
    avg_loss = loss_test / dataset_sizes_test['test']
    avg_acc = acc_test / dataset_sizes_test['test']
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)

In [66]:
dataset_sizes_test

In [67]:
if use_gpu:
    vgg16.cuda() #.cuda() will move everything to the GPU side
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.0005, momentum = 0.9, weight_decay=0.0005)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [68]:
print("Test before training")
eval_model(vgg16, criterion)

In [69]:
visualize_model(vgg16)

In [72]:
#code adapted from https://www.kaggle.com/code/carloalbertobarbano/vgg16-transfer-learning-pytorch/notebook
def train_model(vgg16, criterion, optimizer, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg16.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])
    
    train_loss = []
    train_accuracy = []

    val_loss = []
    val_accuracy = []

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg16.train(True)
        
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)
                
            # Use half training dataset
            if i >= train_batches / 2:
                break
                
            inputs, labels = data
            
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad()
            
            outputs = vgg16(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train * 2 / dataset_sizes[TRAIN]
        train_loss.append(avg_loss)
        
        avg_acc = acc_train * 2 / dataset_sizes[TRAIN]
        train_accuracy.append(avg_acc)
        
        vgg16.train(False)
        vgg16.eval()
            
        for i, data in enumerate(dataloaders[VAL]):
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
                
            inputs, labels = data
            
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda(), )
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad()
            
            outputs = vgg16(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.item()
           
            acc_val += torch.sum(preds == labels.data)
            
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / dataset_sizes[VAL]
        val_loss.append(avg_loss_val)
        
        avg_acc_val = acc_val / dataset_sizes[VAL]
        val_accuracy.append(avg_acc_val)
        
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg16.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    vgg16.load_state_dict(best_model_wts)
    print('train_loss:', train_loss)
    print('\n')
    
    print('train_accuracy: ', train_accuracy)
    print('\n')
    
    print('val_loss: ', val_loss)
    print('\n')
    
    print('val_accuracy: ', val_accuracy)
    print('\n')
    
    return vgg16

In [73]:
vgg16 = train_model(vgg16, criterion, optimizer_ft, num_epochs=30)
torch.save(vgg16.state_dict(), '/kaggle/working/VGG16_v1.pt')

In [74]:
train_loss = [0.02748516672248596, 0.02335947022885543, 0.01974210111572141, 0.01811217683738828, 0.016732405592275703, 0.015311568389775524, 0.014139645742788354, 0.01341035580176459, 0.011762475176252228, 0.011723317290902962, 0.010331566689740867, 0.010638802919779722, 0.009019472943311012, 0.009881487091556061, 0.00911827004924187, 0.008972670997407119, 0.007902410118042236, 0.0077050597658114865, 0.0077709614990459155, 0.006934247788133962, 0.007145934540228399, 0.007054845722529779, 0.006794078764604563, 0.006013630341916211, 0.006228376758471845, 0.005691696616637735, 0.006041063968294149, 0.005835156281312664, 0.005724024996676407, 0.005775055662790189]

val_loss = [0.018836259461005683, 0.02244757811007615, 0.012502466213805836, 0.014264933862233035, 0.012482399723867006, 0.013933113247992497, 0.010542782529190733, 0.010296113010693603, 0.012856502544469189, 0.007498151861559718, 0.008467840510036224, 0.0053892435022599005, 0.006481106927922183, 0.005383909130575061, 0.0072483539003718395, 0.00682119063597776, 0.014194695869521256, 0.009989471186938867, 0.015853084967463637, 0.004023044853760786, 0.004532191415000878, 0.00407760762075176, 0.006640109273409158, 0.004707408756086001, 0.010461586146095572, 0.0053940281744453275, 0.006043445364998823, 0.004451056215451488, 0.0037933143882961256, 0.008085817617974155]


In [148]:
epochs = [i for i in range(1, 31)]
plt.plot(epochs, train_loss, '-o', color='blue', label='train_loss')
plt.plot(epochs, val_loss, '-o', color='green', label='validation_loss')
plt.xticks(np.arange(min(epochs), max(epochs)+1, 1.0))
plt.xlabel('Epochs', size=13)
plt.ylabel('Losses', size=13)
plt.grid(color='#EAE4E3')
plt.xticks(rotation=90)
plt.title('Training and validation losses of VGG16', size=13)
plt.legend()
plt.savefig('../working/vgg16-tv-losses.png', dpi=600,  bbox_inches="tight")
plt.show()

In [92]:
train_accuracy=  [0.6730, 0.7228, 0.7686, 0.7926,0.8091, 0.8222, 0.8432, 0.8500, 0.8693,0.8712, 0.8857, 0.8775, 0.9000, 0.8922,
                  0.8962, 0.9043, 0.9143, 0.9160, 0.9146, 0.9275, 0.9223, 0.9273, 0.9278, 0.9383, 0.9316, 0.9410, 0.9318, 0.9376,
                  0.9390, 0.9358]

validation_accuracy = [0.7861, 0.7413, 0.8455, 0.8294, 0.8469, 0.8425, 0.8739, 0.8880, 0.8600, 0.9091,0.8947, 0.9336, 0.9266, 0.9378, 
                       0.9097, 0.9286, 0.8566, 0.8886, 0.8502, 0.9511, 0.9450, 0.9525, 0.9244, .9419, 0.9008, 0.9366, 0.9236, 0.9469, 
                       0.9558, 0.9216, ]

In [149]:
plt.rcParams['figure.figsize'] = [8, 8]
epochs = [i for i in range(1, 31)]
plt.plot(epochs, train_accuracy, '-o', color='blue', label='train_loss')
plt.plot(epochs, validation_accuracy, '-o', color='green', label='validation_loss')
plt.xticks(np.arange(min(epochs), max(epochs)+1, 1.0))
plt.xlabel('Epochs', size=13)
plt.ylabel('Accuracies', size=13)
plt.grid(color='#EAE4E3')
plt.xticks(rotation=90)
plt.title('Training and validation accuracies of VGG16', size=13)
plt.legend()
plt.savefig('../working/vgg16-tv-accuracies.png', dpi=600,  bbox_inches="tight")
plt.show()

### Visualizing the confusion matrix of the model

In [109]:
predictions, targets = [], []  #code adapted from https://stackoverflow.com/questions/63647547/how-to-find-confusion-matrix-and-plot-it-for-image-classifier-in-pytorch
for images, labels in test_dataloader:
    images, labels = images.cuda(), labels.cuda()
    logps = vgg16(images)
    output = torch.exp(logps)
    pred = torch.argmax(output, 1)

    # convert to numpy arrays
    pred = pred.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    for i in range(len(pred)):
        predictions.append(pred[i])
        targets.append(labels[i])

In [117]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
%matplotlib inline

vgg16 = vgg16.cuda()

In [116]:
classes = image_datasets[TRAIN].classes
classes

In [118]:
cf_matrix = confusion_matrix(targets, predictions)
cf_matrix

In [119]:
class_names = classes
dataframe = pd.DataFrame(cf_matrix, index=class_names, columns=class_names)
dataframe

In [140]:
plt.figure(figsize=(6, 6))  #change size to 4, 4?

#Create heatmap
sns.heatmap(dataframe, annot=True, cbar=True,cmap="OrRd",fmt="d") #'purples', 'PuRd'
plt.title("Confusion Matrix for VGG16 model", size=12), plt.tight_layout()
 
plt.ylabel("Actual Class", size=12), 
plt.xlabel("Predicted Class",  size=12)
plt.tight_layout()
plt.savefig('../working/cmatrxvgg16.png', dpi=600,  bbox_inches="tight")
plt.show()

In [122]:
print("Test Accuracy : {}".format(accuracy_score(targets, predictions)))
print("\nConfusion Matrix : ")
print(confusion_matrix(targets, predictions))
print("\nClassification Report :")
print(classification_report(targets, predictions, target_names=classes))